kütüphanelerimiz


In [ ]:
!pip install tsfresh
!pip install sklearn
!pip install feature-engine

In [360]:
import yfinance
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np
from io import StringIO
import os
from tsfresh import extract_features
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

genel fonksiyonlar

In [ ]:
def fetch_sectors_names():
    url = "https://stockanalysis.com/stocks/industry/sectors/"
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")
        df = pd.read_html(StringIO(str(soup.find_all("table"))))[0]
        return df
    else:
        print(f"Error: Failed to fetch data from page {url}")
        return pd.DataFrame()

def fetch_industry_names():
    url = "https://stockanalysis.com/stocks/industry/all/"
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")
        df = pd.read_html(StringIO(str(soup.find_all("table"))))[0]
        return df
    else:
        print(f"Error: Failed to fetch data from page {url}")
        return pd.DataFrame()

def fetch_data(sectors):
    url = f"https://stockanalysis.com/stocks/sector/{sectors}/"
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")
        df = pd.read_html(str(soup.find_all("table")))[0]
        df.drop(columns='No.', inplace=True)
        return df
    else:
        print(f"Error: Failed to fetch data from page {url}")
        return pd.DataFrame()


def get_rolling_ret(data, n):
    return data.rolling(n).apply(np.prod)

istediğimiz sektorler ile çalışma

In [274]:
dosya_yolu = [ 'stock_sectors/financials.csv', 'stock_sectors/healthcare.csv', 'stock_sectors/technology.csv' ]
sektor = [ 'financials', 'healthcare', 'technology' ]

yukarıda belirtilen dosya ve dosya yolları ile veriyi çeken kod

In [ ]:
for i in range(0,len(dosya_yolu)):
  if os.path.exists(dosya_yolu[i])==False:
    fetch_data(sectors=sektor[i]).to_csv(f'{dosya_yolu[i]}')

sektorlerin tagleri

In [ ]:
sektor_tag = ["f","h","t"]

ihtiyacımız olan verileri hem indirip hem de veri temizliği yapan kısım bazı gereksiz işler yapmışım sonradan kullanmadığım yerler oldu

In [ ]:
for x in range (0,len(dosya_yolu)):
    veriler = pd.read_csv(f"{dosya_yolu[x]}")
    data = yfinance.download(list(veriler.Symbol.dropna()), start='2005-01-01')
    data_close=data['Adj Close'].resample('M').last().pct_change()+1

    lst1,lst2,lst3,lst4 = [],[],[],[]
    veri2 = pd.DataFrame()

    for i in range(0,len(data_close.columns)):
        lst = []
        veri1 = data_close.iloc[:,i:i+1]
        for a in range(0,len(veri1)):
            lst.append(data_close.columns[i])
        veri1.loc[:,"id"] = lst
        veri1.columns = ["close","id"]
        veri2 = pd.concat([veri2,veri1],axis=0)

    veri3 = get_rolling_ret(data_close, 3).melt()
    for i in range(0,len(veri3)):
      lst3.append(veri3.iloc[i:i+1].values)

    for i in range(0,len(lst3)):
      lst4.append(lst3[i][0][1])
    for i in range(0,len(veri2)):
        lst2.append(f"{sektor[x]}")
    veri2.loc[:,"momentum"] = lst4
    veri2.loc[:,"sektor"] = lst2
    veri2 = veri2.dropna()
    veri2.to_csv(f'data/{sektor_tag[x]}.csv')

tsfresh uygulama kısmı

In [259]:
if __name__ == '__main__':
    finance = pd.read_csv("data/f.csv")
    health = pd.read_csv("data/h.csv")
    technology = pd.read_csv("data/t.csv")
    i = 10000
    df = pd.concat([finance.iloc[:i], health.iloc[:i], technology.iloc[:i]])


    print(len(df))
    date = df.iloc[:,0:1]
    close = df.iloc[:,1:2]
    id = df.iloc[:,2:3]
    momentum = df.iloc[:,3:4]
    sektor = df.iloc[:,4:5]
    df = pd.concat([date,close,momentum,id],axis=1)
    exf = extract_features(df, column_id='id', column_sort='Date', n_jobs=1, chunksize=1000)
    exf.to_csv("data/tsfresh.csv")
    print(exf.dropna())

30000


Feature Extraction: 100%|██████████| 1/1 [00:52<00:00, 52.30s/it]


Empty DataFrame
Columns: [close__variance_larger_than_standard_deviation, close__has_duplicate_max, close__has_duplicate_min, close__has_duplicate, close__sum_values, close__abs_energy, close__mean_abs_change, close__mean_change, close__mean_second_derivative_central, close__median, close__mean, close__length, close__standard_deviation, close__variation_coefficient, close__variance, close__skewness, close__kurtosis, close__root_mean_square, close__absolute_sum_of_changes, close__longest_strike_below_mean, close__longest_strike_above_mean, close__count_above_mean, close__count_below_mean, close__last_location_of_maximum, close__first_location_of_maximum, close__last_location_of_minimum, close__first_location_of_minimum, close__percentage_of_reoccurring_values_to_all_values, close__percentage_of_reoccurring_datapoints_to_all_datapoints, close__sum_of_reoccurring_values, close__sum_of_reoccurring_data_points, close__ratio_value_number_to_time_series_length, close__sample_entropy, close__m

gereksiz featurelarımızı temizleyeceğiz.

In [ ]:
df = pd.read_csv("data/tsfresh.csv")
print(len(df.columns))
silinecek_kolonlar = []
for i in range(0,len(df.columns)):
  if df.iloc[0:1,i:i+1].values == df.iloc[1:2,i:i+1].values == df.iloc[2:3,i:i+1].values or df.iloc[0:3,i:i+1].isna().all().values  :
    silinecek_kolonlar.append(i)
print(silinecek_kolonlar)
df.drop(df.columns[silinecek_kolonlar], axis=1, inplace=True)
print(len(df.columns))
df.to_csv("data/bosverilersilindi.csv")

son sütuna sektor bilgilerini giriyoruz

In [308]:
df = pd.read_csv("data/bosverilersilindi.csv")
finance = pd.read_csv("data/f.csv")
health = pd.read_csv("data/h.csv")
technology = pd.read_csv("data/t.csv")

lst = []
for i in range(0,len(df)):

  if df.iloc[i:i+1,1:2].values in finance.id.values:
    lst.append("f")
  if df.iloc[i:i+1,1:2].values in health.id.values:
    lst.append("h")
  if df.iloc[i:i+1,1:2].values in technology.id.values:
    lst.append("t")
df.loc[:,"sektor"] = lst
df = df.iloc[:,2:]
df = df.fillna(0)
df.to_csv("data/hazirveri.csv")



gereksiz verileri temizlemeyi yaptığımız kod bloğu

In [332]:
df = pd.read_csv("data/hazirveri.csv")
df = df.iloc[:,1:]
x = df.iloc[:,0:1453]
y = df.iloc[:,1453:]
from sklearn.pipeline import Pipeline
X_train, X_test, y_train, y_test = train_test_split(x,y,test_size=0.3, random_state=0)
from feature_engine.selection import SmartCorrelatedSelection
from sklearn.ensemble import RandomForestClassifier

selector=SmartCorrelatedSelection(method='pearson', threshold=0.90,
                                  selection_method='model_performance',
                                  estimator=RandomForestClassifier(random_state=0),
                                  scoring='f1_macro',
                                  cv=5,
                                 )
selector.fit(X_train, y_train.to_numpy().ravel())
selector.transform(X_train)

print(f'Korelasyonu yuksek olan degiskenler : \n {selector.correlated_feature_sets_}')
print(f'Ciakrtilan degiskenler : \n {selector.features_to_drop_}')
drops = selector.features_to_drop_
df.drop(columns=drops,inplace=True)
df.to_csv("data/droped.csv")

Korelasyonu yuksek olan degiskenler : 
 [{'close__fft_coefficient__attr_"imag"__coeff_97', 'momentum__agg_linear_trend__attr_"slope"__chunk_len_5__f_agg_"var"', 'momentum__fft_coefficient__attr_"real"__coeff_64', 'momentum__fft_coefficient__attr_"real"__coeff_70', 'momentum__fft_coefficient__attr_"real"__coeff_76', 'close__fft_coefficient__attr_"imag"__coeff_67', 'close__fft_coefficient__attr_"abs"__coeff_93', 'momentum__fft_coefficient__attr_"abs"__coeff_12', 'momentum__fft_coefficient__attr_"real"__coeff_14', 'close__fft_coefficient__attr_"real"__coeff_54', 'momentum__fft_coefficient__attr_"abs"__coeff_76', 'momentum__fft_coefficient__attr_"imag"__coeff_1', 'close__fft_coefficient__attr_"abs"__coeff_26', 'close__fft_coefficient__attr_"abs"__coeff_29', 'momentum__fft_coefficient__attr_"real"__coeff_30', 'close__fft_coefficient__attr_"abs"__coeff_96', 'close__fft_coefficient__attr_"real"__coeff_28', 'close__fft_coefficient__attr_"real"__coeff_56', 'close__agg_linear_trend__attr_"stderr

verileri scale etme , parçalama ve svc eğitimi

In [411]:
sc = StandardScaler()
x = df.iloc[:,:-1]
y = df.iloc[:,-1:]
print(df.shape)
x=sc.fit_transform(x)
X_train, X_test, y_train, y_test = train_test_split(x,y,test_size=0.3, random_state=0)
svc = SVC()
svc.fit(X_train,y_train)
predict = svc.predict(X_test)


(260, 92)


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


svc için sonuçlarımız

In [428]:
from sklearn.metrics import f1_score,accuracy_score,recall_score,precision_score
f1 = f1_score(y_test, predict,average='macro')
acc= accuracy_score(y_test, predict)
recall = recall_score(y_test, predict,average='weighted')
pre = precision_score(y_test, predict,average='macro')
print("f1",f1)
print("acc",acc)
print("recall",recall)
print("precision",pre)
from joblib import dump
nesne = svc
dump(nesne, 'svc.joblib')

f1 0.5679012345679012
acc 0.5769230769230769
recall 0.5769230769230769
precision 0.568648876619891


['svc.joblib']

knn eğitimi ve sonuçları

In [427]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn.fit(X_train,y_train)
predict = knn.predict(X_test)

f1 = f1_score(y_test, predict,average='macro')
acc= accuracy_score(y_test, predict)
recall = recall_score(y_test, predict,average='weighted')
pre = precision_score(y_test, predict,average='macro')
print("f1",f1)
print("acc",acc)
print("recall",recall)
print("precision",pre)

from joblib import dump
nesne = knn
dump(nesne, 'knn.joblib')

f1 0.5679012345679012
acc 0.5769230769230769
recall 0.5769230769230769
precision 0.568648876619891


/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


['knn.joblib']

Naive Bayes

In [425]:
from sklearn.naive_bayes import GaussianNB
naive = GaussianNB()
naive.fit(X_train,y_train)
predict = naive.predict(X_test)

f1 = f1_score(y_test, predict,average='macro')
acc= accuracy_score(y_test, predict)
recall = recall_score(y_test, predict,average='weighted')
pre = precision_score(y_test, predict,average='macro')
print("f1",f1)
print("acc",acc)
print("recall",recall)
print("precision",pre)

nesne = naive
dump(nesne, 'naive.joblib')

f1 0.40492361328092885
acc 0.46153846153846156
recall 0.46153846153846156
precision 0.4090909090909091


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


['naive.joblib']

decision tree

Decision Tree Classifier

In [424]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier()
dtc.fit(X_train,y_train)
predict = dtc.predict(X_test)

f1 = f1_score(y_test, predict,average='macro')
acc= accuracy_score(y_test, predict)
recall = recall_score(y_test, predict,average='weighted')
pre = precision_score(y_test, predict,average='macro')
print("f1",f1)
print("acc",acc)
print("recall",recall)
print("precision",pre)

nesne = dtc
dump(nesne, 'dtc.joblib')

f1 0.38616080302126815
acc 0.3974358974358974
recall 0.3974358974358974
precision 0.4143692564745196


['dtc.joblib']

Random Forest

In [426]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
rfc.fit(X_train,y_train)
predict = rfc.predict(X_test)

f1 = f1_score(y_test, predict,average='macro')
acc= accuracy_score(y_test, predict)
recall = recall_score(y_test, predict,average='weighted')
pre = precision_score(y_test, predict,average='macro')
print("f1",f1)
print("acc",acc)
print("recall",recall)
print("precision",pre)

nesne = rfc
dump(nesne, 'rfc.joblib')

<ipython-input-426-7266e28b5073>:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rfc.fit(X_train,y_train)


f1 0.6244343891402716
acc 0.6410256410256411
recall 0.6410256410256411
precision 0.6523835641482699


['rfc.joblib']